In [1]:
!pip install Spotipy

In [1]:
import os
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
import seaborn as sns
%config InlineBackend.figure_format ='retina'
import spotipy
import spotipy.util as util
from spotipy.oauth2 import SpotifyClientCredentials
from spotipy import oauth2
import random
from functools import reduce

import json, os
import traceback
import matplotlib.pyplot as plt
%matplotlib inline



In [2]:
from groupify.app.spotify_utils import get_user_top_tracks, fetch_audio_features, get_playlists_of_user, fetch_playlist_tracks 
from groupify.app.spotify_utils import create_playlist_new, enrich_playlist, fetch_playlist_tracks

In [3]:
# constants
redirect_uri = 'https://example.com'

In [4]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy.util as util

import groupify.app.psecrets as psecrets

try: 
    cid = psecrets.client_id
    secret = psecrets.secret
    userdata = json.load(open('groupify/app/json/userdata.json', 'r'))
    username = list(userdata.keys())[0]
    scope = 'user-top-read user-read-recently-played user-follow-read playlist-modify-public user-library-read playlist-read-collaborative'
    token = util.prompt_for_user_token(username, scope, client_id=cid, client_secret=secret, redirect_uri=redirect_uri)

    if token:
        sp = spotipy.Spotify(auth=token)
    else:
        print("Can't get token for", username)
except: 
    print(traceback.format_exc(5))


In [5]:
df_favorite = get_user_top_tracks(sp)
df_fav = fetch_audio_features (sp, df_favorite)


<class 'list'>


In [6]:
df_fav.info()

<class 'pandas.core.frame.DataFrame'>
Index: 50 entries, 6tFzWFRM5n8GyMLVvzzjUB to 145BjTry1UkM3WL74UEqtt
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   danceability      50 non-null     float64
 1   acousticness      50 non-null     float64
 2   energy            50 non-null     float64
 3   tempo             50 non-null     float64
 4   instrumentalness  50 non-null     float64
 5   loudness          50 non-null     float64
 6   liveness          50 non-null     float64
 7   duration_ms       50 non-null     int64  
 8   key               50 non-null     int64  
 9   valence           50 non-null     float64
 10  speechiness       50 non-null     float64
 11  mode              50 non-null     int64  
dtypes: float64(9), int64(3)
memory usage: 5.1+ KB


In [7]:
df_fav.hist(figsize=(35,25)) 
plt.tight_layout()
plt.show()

/Users/tejasbot/anaconda3/lib/python3.8/site-packages/pandas/plotting/_matplotlib/tools.py:331: MatplotlibDeprecationWarning: 
The is_first_col function was deprecated in Matplotlib 3.4 and will be removed two minor releases later. Use ax.get_subplotspec().is_first_col() instead.
  if ax.is_first_col():


In [8]:
df_playlists = get_playlists_of_user(sp, username)
df_playlists

id                                               name  \
0   2ZSR6zt1YF8FBdfuvw6KNc                                        Rudra Veena   
1   4LJMgDqw8LBDZ5ERzxfQ0g                                    Fanna Fi Allah    
2   0DcztO02Tox6mIEGfbYJIr                                          JS Blend2   
3   1kgaY77cpRDGhcYmVNHvdV                                            karaoke   
4   72GrzHx43yw0nHa3BuAPrL  Craig Pruess and the Singers of the Art of Liv...   
5   3PTg0gKdhPv9TPJmw60KSz                               Shankar - Raga Aberi   
6   1rTvmcaTynZIg6VqGxjphk  Various Artists - The Best Of Carnatic Vocal, ...   
7   6n6jIjZy2Vg9VCMx6Swv6y                          Various Artists - Varnams   
8   2OFmUyEWo5LaCfEMv5zZdR                                            Starred   
9   7AR1r6RMIxZzfXp1JBvAhj     The World Players – The World Of Celtic Fiddle   
10  1KkrVONQgCv9ijVE3j9NqS     Satyajit Ray – The Masterworks Of Satyajit Ray   
11  5RAFOgwi4AtumrEG7KeHWH                             The Darjeeling Limited   
12  6ZSuc73ggKacdq50SxbZwJ  Antonio Vivaldi – 100 Pieces of Classical Musi...   
13  5DCbt84RG5uf2DbRTzybT0                 Saashwathi Prabhu – Mystic Journey   
14  2IWxWGVN1vz4DMdh3lrKYZ  Moodagent - Lehar Remix - Raag Desh by Rakesh ...   
15  0XcJjckAXfv02QfQkqFkXr  Zakir Hussain – Rhythmic Impressions Of Ustad ...   
16  7nFRkJjRbA7YkOBZKhpw7j                                               Work   

    #tracks  
0         9  
1         3  
2         0  
3         1  
4         6  
5         8  
6        14  
7        22  
8         3  
9        26  
10        8  
11       22  
12      100  
13       11  
14       25  
15       13  
16       68

In [9]:
from groupify.app.spotify_utils import mean_audio_features_playlist
# Merge them together
import numpy
dataframes = []
# Loop through the filenames to populate dataframes with different dataframes 
for  playlist in df_playlists['id']:
    try:
        mean_df = mean_audio_features_playlist(sp, playlist)

        if any(numpy.isnan(mean_df.values)): 
            
            print ("Skip "+playlist)
            continue
        dataframes.append(mean_df)
    except:
        print ("Skip "+playlist)

Skip 0DcztO02Tox6mIEGfbYJIr
Skip 7nFRkJjRbA7YkOBZKhpw7j


In [10]:
dataframes

[                  2ZSR6zt1YF8FBdfuvw6KNc
 danceability                3.502222e-01
 acousticness                7.993333e-01
 energy                      2.970667e-01
 tempo                       1.195447e+02
 instrumentalness            5.196667e-01
 loudness                   -1.946933e+01
 liveness                    2.737222e-01
 duration_ms                 1.191608e+06
 key                         4.111111e+00
 valence                     2.755333e-01
 speechiness                 4.655556e-02
 mode                        8.888889e-01,
                   4LJMgDqw8LBDZ5ERzxfQ0g
 danceability                    0.401333
 acousticness                    0.056270
 energy                          0.740333
 tempo                         115.697000
 instrumentalness                0.351947
 loudness                       -9.042667
 liveness                        0.540000
 duration_ms                886124.333333
 key                             4.333333
 valence                         

In [11]:
X = reduce(lambda left,right: pd.merge(left,right, left_index=True, right_index=True), dataframes)
X

2ZSR6zt1YF8FBdfuvw6KNc  4LJMgDqw8LBDZ5ERzxfQ0g  \
danceability                3.502222e-01                0.401333   
acousticness                7.993333e-01                0.056270   
energy                      2.970667e-01                0.740333   
tempo                       1.195447e+02              115.697000   
instrumentalness            5.196667e-01                0.351947   
loudness                   -1.946933e+01               -9.042667   
liveness                    2.737222e-01                0.540000   
duration_ms                 1.191608e+06           886124.333333   
key                         4.111111e+00                4.333333   
valence                     2.755333e-01                0.727333   
speechiness                 4.655556e-02                0.059233   
mode                        8.888889e-01                0.666667   

                  1kgaY77cpRDGhcYmVNHvdV  72GrzHx43yw0nHa3BuAPrL  \
danceability                      0.4440                0.201517   
acousticness                      0.7350                0.916000   
energy                            0.5910                0.170167   
tempo                            82.9520               98.088667   
instrumentalness                  0.0000                0.002140   
loudness                        -10.0020              -16.691667   
liveness                          0.0734                0.123450   
duration_ms                  271581.0000           686155.333333   
key                               1.0000                5.000000   
valence                           0.5870                0.129333   
speechiness                       0.0637                0.035467   
mode                              1.0000                0.833333   

                  3PTg0gKdhPv9TPJmw60KSz  1rTvmcaTynZIg6VqGxjphk  \
danceability                    0.476125                0.392143   
acousticness                    0.520666                0.819929   
energy                          0.503000                0.525071   
tempo                          85.087875              137.577071   
instrumentalness                0.765750                0.017584   
loudness                      -13.549250              -11.447143   
liveness                        0.201500                0.175486   
duration_ms                364161.625000           577384.785714   
key                             9.500000                5.785714   
valence                         0.579125                0.475643   
speechiness                     0.056012                0.093371   
mode                            0.250000                0.785714   

                  6n6jIjZy2Vg9VCMx6Swv6y  2OFmUyEWo5LaCfEMv5zZdR  \
danceability                    0.416909            5.193333e-01   
acousticness                    0.743591            6.976667e-01   
energy                          0.603455            6.176667e-01   
tempo                         138.897864            1.657390e+02   
instrumentalness                0.035878            1.668022e-01   
loudness                       -9.954318           -8.768667e+00   
liveness                        0.211655            1.573333e-01   
duration_ms                427728.272727            1.273367e+06   
key                             6.318182            5.000000e+00   
valence                         0.651364            4.746667e-01   
speechiness                     0.163409            1.095667e-01   
mode                            0.863636            6.666667e-01   

                  7AR1r6RMIxZzfXp1JBvAhj  1KkrVONQgCv9ijVE3j9NqS  \
danceability                    0.319973                0.312750   
acousticness                    0.878500                0.933375   
energy                          0.193335                0.293625   
tempo                          94.847115              125.991625   
instrumentalness                0.318358                0.496907   
loudness                      -18.276115              -16.181875   
li

In [12]:
Y = pd.DataFrame(df_fav.median(), columns= ['fav_playlist'])
Y= Y.drop('mode')
Y

fav_playlist
danceability            0.3560
acousticness            0.8835
energy                  0.3230
tempo                 113.8445
instrumentalness        0.2040
loudness              -12.8645
liveness                0.1255
duration_ms        253373.5000
key                     5.0000
valence                 0.3135
speechiness             0.0410
mode                    1.0000

In [13]:
# Analyze feature importances
from sklearn.ensemble._forest import RandomForestRegressor
# Can combine step above with this
forest = RandomForestRegressor(n_estimators=10)#random_state=42, max_depth=2, max_features=9) 
forest.fit(X,Y.values.ravel())
importances = forest.feature_importances_
indices = np.argsort(importances)[::-1]
# Print the feature rankings
print("Playlist ranking:")
  
for f in range(len(importances)):
    playlist_name = df_playlists[df_playlists['id']==X.columns[indices[f]]]['name'].values[0]
    
    print("%d. %s %s %f " % (f + 1, 
            X.columns[indices[f]],
            playlist_name,
            importances[indices[f]]))

Playlist ranking:
1. 1rTvmcaTynZIg6VqGxjphk Various Artists - The Best Of Carnatic Vocal, Vol. 1 & 2 0.200001 
2. 2OFmUyEWo5LaCfEMv5zZdR Starred 0.197893 
3. 1kgaY77cpRDGhcYmVNHvdV karaoke 0.100268 
4. 5DCbt84RG5uf2DbRTzybT0 Saashwathi Prabhu – Mystic Journey 0.100033 
5. 2ZSR6zt1YF8FBdfuvw6KNc Rudra Veena 0.100005 
6. 0XcJjckAXfv02QfQkqFkXr Zakir Hussain – Rhythmic Impressions Of Ustad Zakir Hussain 0.100000 
7. 4LJMgDqw8LBDZ5ERzxfQ0g Fanna Fi Allah  0.099992 
8. 3PTg0gKdhPv9TPJmw60KSz Shankar - Raga Aberi 0.091186 
9. 6ZSuc73ggKacdq50SxbZwJ Antonio Vivaldi – 100 Pieces of Classical Music For Your Brain 0.009687 
10. 6n6jIjZy2Vg9VCMx6Swv6y Various Artists - Varnams 0.000461 
11. 72GrzHx43yw0nHa3BuAPrL Craig Pruess and the Singers of the Art of Living - Sacred Chants of Shiva 0.000458 
12. 7AR1r6RMIxZzfXp1JBvAhj The World Players – The World Of Celtic Fiddle 0.000008 
13. 5RAFOgwi4AtumrEG7KeHWH The Darjeeling Limited 0.000006 
14. 2IWxWGVN1vz4DMdh3lrKYZ Moodagent - Lehar Remix - Raag D

In [14]:
for i, playlist in enumerate(df_playlists['id']):
    try:
        string_command = "df_{} = fetch_audio_features(sp, playlist)".format(playlist)
        print("Create {}".format(string_command))
        exec(string_command)
    except:
        print("playlist with id {} is not valid, skiping ".format(playlist))
        pass


frames = []
for i, playlist in enumerate(df_playlists['id']):
    try:
        string_command = "frames.append(df_{})".format(playlist)
        print("Create {}".format(string_command))
        exec(string_command)
    except:
        print("playlist with id {} is not valid, skiping ".format(playlist))
        pass
print (frames)
df =pd.concat(frames)
df.drop_duplicates()
df = df.set_index(['track_id'])



Create df_2ZSR6zt1YF8FBdfuvw6KNc = fetch_audio_features(sp, playlist)
Create df_4LJMgDqw8LBDZ5ERzxfQ0g = fetch_audio_features(sp, playlist)
Create df_0DcztO02Tox6mIEGfbYJIr = fetch_audio_features(sp, playlist)
Create df_1kgaY77cpRDGhcYmVNHvdV = fetch_audio_features(sp, playlist)
Create df_72GrzHx43yw0nHa3BuAPrL = fetch_audio_features(sp, playlist)
Create df_3PTg0gKdhPv9TPJmw60KSz = fetch_audio_features(sp, playlist)
Create df_1rTvmcaTynZIg6VqGxjphk = fetch_audio_features(sp, playlist)
Create df_6n6jIjZy2Vg9VCMx6Swv6y = fetch_audio_features(sp, playlist)
Create df_2OFmUyEWo5LaCfEMv5zZdR = fetch_audio_features(sp, playlist)
Create df_7AR1r6RMIxZzfXp1JBvAhj = fetch_audio_features(sp, playlist)
Create df_1KkrVONQgCv9ijVE3j9NqS = fetch_audio_features(sp, playlist)
Create df_5RAFOgwi4AtumrEG7KeHWH = fetch_audio_features(sp, playlist)
Create df_6ZSuc73ggKacdq50SxbZwJ = fetch_audio_features(sp, playlist)
Create df_5DCbt84RG5uf2DbRTzybT0 = fetch_audio_features(sp, playlist)
Create df_2IWxWGVN1v

KeyError: "None of ['track_id'] are in the columns"

In [15]:
df

danceability  acousticness  energy    tempo  \
track_id                                                              
1v7hb0IjIyzmeaDJy6lsB5         0.242         0.962  0.0326  113.674   
6Ek6YcYWgpDctbMgnShcfG         0.194         0.906  0.1870  128.961   
4THJEw7GSeFCBoWxLbWdPC         0.333         0.497  0.5250  142.456   
0a4OxCTdwnJ8P2ni1u4yV5         0.306         0.888  0.2360   88.276   
3pojxH8ZwVWPv71O7xoHtp         0.481         0.768  0.3210  103.676   
...                              ...           ...     ...      ...   
64Wf0OnCvdq2uVxrUCJOsq         0.499         0.133  0.8390  141.027   
14EYVKNIgesnKeaq92b8Zd         0.509         0.481  0.8910  139.575   
5dJEx7XVIMB8H5yhrxzJkd         0.461         0.898  0.7340  148.379   
5KIYWWfLUVmtNGZY2upgqy         0.484         0.844  0.6750  112.305   
6OLgT4oP4QDByd01tXeQ0M         0.484         0.037  0.4490   92.913   

                        instrumentalness  loudness  liveness duration_ms key  \
track_id                                                                       
1v7hb0IjIyzmeaDJy6lsB5             0.635   -23.106    0.1050     1476440   9   
6Ek6YcYWgpDctbMgnShcfG             0.511   -14.261    0.6510     2212747   9   
4THJEw7GSeFCBoWxLbWdPC             0.784   -11.218    0.6880     1056120   4   
0a4OxCTdwnJ8P2ni1u4yV5             0.487   -19.389    0.0645     1418560   0   
3pojxH8ZwVWPv71O7xoHtp             0.561   -22.384    0.0657      920067   2   
...                                  ...       ...       ...         ...  ..   
64Wf0OnCvdq2uVxrUCJOsq             0.765   -22.837    0.0883      191797   1   
14EYVKNIgesnKeaq92b8Zd             0.905   -22.596    0.0564      219707   8   
5dJEx7XVIMB8H5yhrxzJkd             0.881   -15.966    0.0974      212555   1   
5KIYWWfLUVmtNGZY2upgqy             0.850   -18.457    0.0728     1016105   1   
6OLgT4oP4QDByd01tXeQ0M             0.878   -23.830    0.5880     1912529   9   

                        valence  speechiness mode  
track_id                                           
1v7hb0IjIyzmeaDJy6lsB5   0.0473       0.0378    1  
6Ek6YcYWgpDctbMgnShcfG   0.0394       0.0386    1  
4THJEw7GSeFCBoWxLbWdPC   0.2460       0.0476    1  
0a4OxCTdwnJ8P2ni1u4yV5   0.0811       0.0362    1  
3pojxH8ZwVWPv71O7xoHtp   0.3820       0.0460    0  
...                         ...          ...  ...  
64Wf0OnCvdq2uVxrUCJOsq   0.9580       0.1170    1  
14EYVKNIgesnKeaq92b8Zd   0.6590       0.0784    1  
5dJEx7XVIMB8H5yhrxzJkd   0.6250       0.0739    1  
5KIYWWfLUVmtNGZY2upgqy   0.7480       0.1080    1  
6OLgT4oP4QDByd01tXeQ0M   0.3270       0.0543    0  

[271 rows x 12 columns]

In [16]:
Y= Y.squeeze('columns')
Y = Y.sort_index( axis=0, level=None, ascending=True, inplace=False, kind='quicksort')
Y

acousticness             0.8835
danceability             0.3560
duration_ms         253373.5000
energy                   0.3230
instrumentalness         0.2040
key                      5.0000
liveness                 0.1255
loudness               -12.8645
mode                     1.0000
speechiness              0.0410
tempo                  113.8445
valence                  0.3135
Name: fav_playlist, dtype: float64

In [29]:
std_dev = pd.DataFrame(df_fav.std(), columns= ['fav_playlist'])
std_dev

fav_playlist
danceability           0.161335
acousticness           0.258822
energy                 0.233836
tempo                 28.651836
instrumentalness       0.413918
loudness               5.159187
liveness               0.210318
duration_ms       231543.581909
key                    3.432676
valence                0.258780
speechiness            0.030075
mode                   0.370328

In [17]:
# Subtract mean of the favorite plalyist from the top 3 playlist 
df1= df.subtract(Y, axis='columns') 
df1

acousticness  danceability  duration_ms  energy  \
track_id                                                                  
1v7hb0IjIyzmeaDJy6lsB5        0.0785        -0.114  1.22307e+06 -0.2904   
6Ek6YcYWgpDctbMgnShcfG        0.0225        -0.162  1.95937e+06 -0.1360   
4THJEw7GSeFCBoWxLbWdPC       -0.3865        -0.023       802746  0.2020   
0a4OxCTdwnJ8P2ni1u4yV5        0.0045        -0.050  1.16519e+06 -0.0870   
3pojxH8ZwVWPv71O7xoHtp       -0.1155         0.125       666694 -0.0020   
...                              ...           ...          ...     ...   
64Wf0OnCvdq2uVxrUCJOsq       -0.7505         0.143     -61576.5  0.5160   
14EYVKNIgesnKeaq92b8Zd       -0.4025         0.153     -33666.5  0.5680   
5dJEx7XVIMB8H5yhrxzJkd        0.0145         0.105     -40818.5  0.4110   
5KIYWWfLUVmtNGZY2upgqy       -0.0395         0.128       762732  0.3520   
6OLgT4oP4QDByd01tXeQ0M       -0.8465         0.128  1.65916e+06  0.1260   

                        instrumentalness key  liveness  loudness mode  \
track_id                                                                
1v7hb0IjIyzmeaDJy6lsB5             0.431   4   -0.0205  -10.2415    0   
6Ek6YcYWgpDctbMgnShcfG             0.307   4    0.5255   -1.3965    0   
4THJEw7GSeFCBoWxLbWdPC             0.580  -1    0.5625    1.6465    0   
0a4OxCTdwnJ8P2ni1u4yV5             0.283  -5   -0.0610   -6.5245    0   
3pojxH8ZwVWPv71O7xoHtp             0.357  -3   -0.0598   -9.5195   -1   
...                                  ...  ..       ...       ...  ...   
64Wf0OnCvdq2uVxrUCJOsq             0.561  -4   -0.0372   -9.9725    0   
14EYVKNIgesnKeaq92b8Zd             0.701   3   -0.0691   -9.7315    0   
5dJEx7XVIMB8H5yhrxzJkd             0.677  -4   -0.0281   -3.1015    0   
5KIYWWfLUVmtNGZY2upgqy             0.646  -4   -0.0527   -5.5925    0   
6OLgT4oP4QDByd01tXeQ0M             0.674   4    0.4625  -10.9655   -1   

                        speechiness    tempo  valence  
track_id                                               
1v7hb0IjIyzmeaDJy6lsB5      -0.0032  -0.1705  -0.2662  
6Ek6YcYWgpDctbMgnShcfG      -0.0024  15.1165  -0.2741  
4THJEw7GSeFCBoWxLbWdPC       0.0066  28.6115  -0.0675  
0a4OxCTdwnJ8P2ni1u4yV5      -0.0048 -25.5685  -0.2324  
3pojxH8ZwVWPv71O7xoHtp       0.0050 -10.1685   0.0685  
...                             ...      ...      ...  
64Wf0OnCvdq2uVxrUCJOsq       0.0760  27.1825   0.6445  
14EYVKNIgesnKeaq92b8Zd       0.0374  25.7305   0.3455  
5dJEx7XVIMB8H5yhrxzJkd       0.0329  34.5345   0.3115  
5KIYWWfLUVmtNGZY2upgqy       0.0670  -1.5395   0.4345  
6OLgT4oP4QDByd01tXeQ0M       0.0133 -20.9315   0.0135  

[271 rows x 12 columns]

In [31]:
## this operation needs to be performed with standard deviation, not median. 
df1= df1.divide(Y, axis='columns') ## not sure if this has to be fixed!
# df1

In [19]:
# df1 = df1.drop(['instrumentalness'], axis=1)

In [20]:
# Add all the score 
df1['variation'] = df1.sum(axis=1)
# take the absoulte of the variatio
df1['variation'] = df1['variation'].abs()
df1 = df1.drop_duplicates()
df1

acousticness  danceability  duration_ms  energy  \
track_id                                                                  
1v7hb0IjIyzmeaDJy6lsB5        0.0785        -0.114  1.22307e+06 -0.2904   
6Ek6YcYWgpDctbMgnShcfG        0.0225        -0.162  1.95937e+06 -0.1360   
4THJEw7GSeFCBoWxLbWdPC       -0.3865        -0.023       802746  0.2020   
0a4OxCTdwnJ8P2ni1u4yV5        0.0045        -0.050  1.16519e+06 -0.0870   
3pojxH8ZwVWPv71O7xoHtp       -0.1155         0.125       666694 -0.0020   
...                              ...           ...          ...     ...   
64Wf0OnCvdq2uVxrUCJOsq       -0.7505         0.143     -61576.5  0.5160   
14EYVKNIgesnKeaq92b8Zd       -0.4025         0.153     -33666.5  0.5680   
5dJEx7XVIMB8H5yhrxzJkd        0.0145         0.105     -40818.5  0.4110   
5KIYWWfLUVmtNGZY2upgqy       -0.0395         0.128       762732  0.3520   
6OLgT4oP4QDByd01tXeQ0M       -0.8465         0.128  1.65916e+06  0.1260   

                        instrumentalness key  liveness  loudness mode  \
track_id                                                                
1v7hb0IjIyzmeaDJy6lsB5             0.431   4   -0.0205  -10.2415    0   
6Ek6YcYWgpDctbMgnShcfG             0.307   4    0.5255   -1.3965    0   
4THJEw7GSeFCBoWxLbWdPC             0.580  -1    0.5625    1.6465    0   
0a4OxCTdwnJ8P2ni1u4yV5             0.283  -5   -0.0610   -6.5245    0   
3pojxH8ZwVWPv71O7xoHtp             0.357  -3   -0.0598   -9.5195   -1   
...                                  ...  ..       ...       ...  ...   
64Wf0OnCvdq2uVxrUCJOsq             0.561  -4   -0.0372   -9.9725    0   
14EYVKNIgesnKeaq92b8Zd             0.701   3   -0.0691   -9.7315    0   
5dJEx7XVIMB8H5yhrxzJkd             0.677  -4   -0.0281   -3.1015    0   
5KIYWWfLUVmtNGZY2upgqy             0.646  -4   -0.0527   -5.5925    0   
6OLgT4oP4QDByd01tXeQ0M             0.674   4    0.4625  -10.9655   -1   

                        speechiness    tempo  valence     variation  
track_id                                                             
1v7hb0IjIyzmeaDJy6lsB5      -0.0032  -0.1705  -0.2662  1.223060e+06  
6Ek6YcYWgpDctbMgnShcfG      -0.0024  15.1165  -0.2741  1.959392e+06  
4THJEw7GSeFCBoWxLbWdPC       0.0066  28.6115  -0.0675  8.027766e+05  
0a4OxCTdwnJ8P2ni1u4yV5      -0.0048 -25.5685  -0.2324  1.165149e+06  
3pojxH8ZwVWPv71O7xoHtp       0.0050 -10.1685   0.0685  6.666702e+05  
...                             ...      ...      ...           ...  
64Wf0OnCvdq2uVxrUCJOsq       0.0760  27.1825   0.6445  6.156214e+04  
14EYVKNIgesnKeaq92b8Zd       0.0374  25.7305   0.3455  3.364617e+04  
5dJEx7XVIMB8H5yhrxzJkd       0.0329  34.5345   0.3115  4.078954e+04  
5KIYWWfLUVmtNGZY2upgqy       0.0670  -1.5395   0.4345  7.627219e+05  
6OLgT4oP4QDByd01tXeQ0M       0.0133 -20.9315   0.0135  1.659127e+06  

[271 rows x 13 columns]

In [21]:
# Now we have the variation, we will take the songs with the least variation 
df2 = df1.nsmallest(50,'variation', keep='first')
df2

acousticness  danceability duration_ms  energy  \
track_id                                                                 
34A7Mhcx9MtYzg1gKelDPH       -0.0125        0.0350       293.5  0.2900   
0iJIu3DQ3pd0UgYErNYrxC       -0.0575        0.1270      1302.5  0.3050   
4ksGuAeANt8PnrIfmPgTRO        0.0435       -0.1520     -2186.5 -0.1850   
4WnxBVhWBcM8z6GRscabSm        0.0925       -0.1530     -2600.5 -0.2450   
0JwSo7FRo1bPRp2IP3kJgM       -0.1165        0.0270      2733.5  0.5970   
6anG7aR0p9enoBrwQybI4A       -0.2595        0.2480      4266.5  0.1080   
6ZilcYUIICcj8ihQ165irs       -0.4115        0.2040      5773.5  0.4650   
0NoooClEgxqlqb4A5NBx9a       -0.1215        0.1210     -8346.5  0.1430   
4wcBlgP2YgOqoXzupVT8rb        0.1125        0.0250     -9173.5 -0.3126   
5YsUCWWe7pdnJO43Yj7TFh       -0.0625        0.0190     14445.5  0.4800   
67VWosPALzIS5jSoOrElFj       -0.1485        0.0880     18207.5  0.2680   
3k0uRcdCErJnwup3sfjUX6        0.1095       -0.0430    -18400.5 -0.3060   
7rZ5cv4pLF5X8EIe9D9jwS        0.0735       -0.2816    -20880.5 -0.3120   
2eRxWvbXjEZYdLW5lErC3T       -0.0385        0.3010     23426.5  0.3820   
5P0WyP7KPYTOZwnFPsqGjQ       -0.0175       -0.1490     24666.5 -0.1660   
7tXLXpjVFwsNbmzgygvA8x        0.0805       -0.1580     24826.5 -0.3115   
2iDlp91egLSxpeuuYeLsqg        0.0755        0.2250    -25280.5 -0.2609   
2ExEChLCSorn4k3FUxs7td       -0.3585        0.1110     25506.5  0.0860   
3sAqCSS70AIBmZitPV0iUX       -0.2995        0.1160    -26186.5  0.1970   
01CWFykGVkAgqaq6mx4S9t        0.0765       -0.2410     26293.5 -0.2617   
3q0wuJHbEtwRX5mVYXAQR5        0.0935       -0.1780    -28693.5 -0.2100   
16fPzsYDNAz5CphU8SNDM2        0.0645        0.1000     29613.5  0.0420   
20YOTskpIGrtIUEcyt16xI       -0.5785        0.2100    -30415.5  0.4190   
7kWxCuG2ReGIMtjLe2CGav       -0.1095        0.0770     30413.5  0.4200   
14EYVKNIgesnKeaq92b8Zd       -0.4025        0.1530    -33666.5  0.5680   
1pVTwIeqARgzKllIyzqpk8        0.0975        0.1120     33893.5 -0.1800   
5xnlsjGhXul8b5dE7HnWg8       -0.3095        0.0920     35179.5  0.1980   
7ecszxDX70AgUT6jNLTB7w       -0.8486        0.2960     37266.5  0.3210   
5dJEx7XVIMB8H5yhrxzJkd        0.0145        0.1050    -40818.5  0.4110   
4fbSQbKoHNvIfqmwvb0xGJ        0.0745        0.1210    -41200.5  0.0820   
4wBDr96bucxRIOzv1wPPd1        0.1075       -0.0810    -42293.5 -0.2839   
7ywbaVjKImedTvUguWQMih        0.1085       -0.0350     43998.5 -0.2130   
5yMnVl9geQTSzsIK2agXpl        0.1055       -0.0400     44540.5 -0.2850   
5CnWRO3c4UWRwCkGxKO0Ly       -0.0035        0.0860    -44973.5 -0.0240   
1et1pAxMY8jFfblk9JIMvX       -0.0215        0.2380    -46106.5 -0.1030   
17TYaKhyzETADo7SOoTTy5       -0.6915        0.2530     46266.5  0.3090   
3Twm0yU9Hv64SVbCenU00c       -0.2515        0.1430    -46390.5  0.3300   
3JQdFzEDxg8YUpxvGpnwxJ       -0.2865        0.1440    -47412.5  0.2180   
2ZE2A47zah0C3Ee2uVKDbT       -0.5925        0.0430    -47760.5  0.4750   
4bevNWkPkIvtQfZMJiysVP        0.1095        0.2580    -49053.5 -0.2464   
1GSKdr9KifUNGGLDfZ6hqh        0.1085        0.0250     49586.5 -0.2933   
3L6bCxrwKe1qnsqFzssR4R       -0.7225        0.1080    -53320.5  0.1090   
1W6CAYXXYGYnq0xFV0CiZb        0.1005       -0.0840     53919.5 -0.0990   
2aiNUE9ia8e9WdUpVIcVYQ        0.0305        0.1920    -54000.5 -0.0610   
4h83R41t8oX0mctk2KLCBZ        0.0515       -0.1080    -54146.5  0.0100   
1DWANIwKlOjnQ3Yn7gJmMe        0.1095       -0.0530    -54306.5 -0.3103   
4yK7u3SViQC3Jhw4XUHkBC       -0.7655        0.1750     55266.5  0.2490   
79NwpxAm72bP8KKS8el0tD       -0.0505        0.1870    -56375.5  0.6320   
28iwfJUUjKbiPEQ9eVx4XH       -0.4115        0.2260     56559.5  0.0520   
2MRWXvnatHSK9S9JfXQQvp       -0.1995        0.2620    -57219.5  0.1520   

                        instrumentalness key  liveness  loudness mode  \
track_id                                                                
34A7Mhcx9MtYzg1gKelDPH       

In [22]:
new_playlist_id = create_playlist_new(sp,  'JS Blend2', 'Test playlist created using python!')

In [23]:
get_playlists_of_user(sp,username)

id                                               name  \
0   5CFcnYnuiiKhfK6TD1jRxS                                          JS Blend2   
1   2ZSR6zt1YF8FBdfuvw6KNc                                        Rudra Veena   
2   4LJMgDqw8LBDZ5ERzxfQ0g                                    Fanna Fi Allah    
3   0DcztO02Tox6mIEGfbYJIr                                          JS Blend2   
4   1kgaY77cpRDGhcYmVNHvdV                                            karaoke   
5   72GrzHx43yw0nHa3BuAPrL  Craig Pruess and the Singers of the Art of Liv...   
6   3PTg0gKdhPv9TPJmw60KSz                               Shankar - Raga Aberi   
7   1rTvmcaTynZIg6VqGxjphk  Various Artists - The Best Of Carnatic Vocal, ...   
8   6n6jIjZy2Vg9VCMx6Swv6y                          Various Artists - Varnams   
9   2OFmUyEWo5LaCfEMv5zZdR                                            Starred   
10  7AR1r6RMIxZzfXp1JBvAhj     The World Players – The World Of Celtic Fiddle   
11  1KkrVONQgCv9ijVE3j9NqS     Satyajit Ray – The Masterworks Of Satyajit Ray   
12  5RAFOgwi4AtumrEG7KeHWH                             The Darjeeling Limited   
13  6ZSuc73ggKacdq50SxbZwJ  Antonio Vivaldi – 100 Pieces of Classical Musi...   
14  5DCbt84RG5uf2DbRTzybT0                 Saashwathi Prabhu – Mystic Journey   
15  2IWxWGVN1vz4DMdh3lrKYZ  Moodagent - Lehar Remix - Raag Desh by Rakesh ...   
16  0XcJjckAXfv02QfQkqFkXr  Zakir Hussain – Rhythmic Impressions Of Ustad ...   
17  7nFRkJjRbA7YkOBZKhpw7j                                               Work   

    #tracks  
0         0  
1         9  
2         3  
3         0  
4         1  
5         6  
6         8  
7        14  
8        22  
9         3  
10       26  
11        8  
12       22  
13      100  
14       11  
15       25  
16       13  
17       68

In [24]:
list_track = df2.index

In [25]:
enrich_playlist(sp, username, new_playlist_id, list_track) #playlist id

In [26]:
get_playlists_of_user(sp,username)

id                                               name  \
0   5CFcnYnuiiKhfK6TD1jRxS                                          JS Blend2   
1   2ZSR6zt1YF8FBdfuvw6KNc                                        Rudra Veena   
2   4LJMgDqw8LBDZ5ERzxfQ0g                                    Fanna Fi Allah    
3   0DcztO02Tox6mIEGfbYJIr                                          JS Blend2   
4   1kgaY77cpRDGhcYmVNHvdV                                            karaoke   
5   72GrzHx43yw0nHa3BuAPrL  Craig Pruess and the Singers of the Art of Liv...   
6   3PTg0gKdhPv9TPJmw60KSz                               Shankar - Raga Aberi   
7   1rTvmcaTynZIg6VqGxjphk  Various Artists - The Best Of Carnatic Vocal, ...   
8   6n6jIjZy2Vg9VCMx6Swv6y                          Various Artists - Varnams   
9   2OFmUyEWo5LaCfEMv5zZdR                                            Starred   
10  7AR1r6RMIxZzfXp1JBvAhj     The World Players – The World Of Celtic Fiddle   
11  1KkrVONQgCv9ijVE3j9NqS     Satyajit Ray – The Masterworks Of Satyajit Ray   
12  5RAFOgwi4AtumrEG7KeHWH                             The Darjeeling Limited   
13  6ZSuc73ggKacdq50SxbZwJ  Antonio Vivaldi – 100 Pieces of Classical Musi...   
14  5DCbt84RG5uf2DbRTzybT0                 Saashwathi Prabhu – Mystic Journey   
15  2IWxWGVN1vz4DMdh3lrKYZ  Moodagent - Lehar Remix - Raag Desh by Rakesh ...   
16  0XcJjckAXfv02QfQkqFkXr  Zakir Hussain – Rhythmic Impressions Of Ustad ...   
17  7nFRkJjRbA7YkOBZKhpw7j                                               Work   

    #tracks  
0        50  
1         9  
2         3  
3         0  
4         1  
5         6  
6         8  
7        14  
8        22  
9         3  
10       26  
11        8  
12       22  
13      100  
14       11  
15       25  
16       13  
17       68

In [28]:
fetch_playlist_tracks(sp, new_playlist_id)

track_id                                         track_name
0   34A7Mhcx9MtYzg1gKelDPH                                Raga Aberi Track 03
1   0iJIu3DQ3pd0UgYErNYrxC                                       Double Dayan
2   4ksGuAeANt8PnrIfmPgTRO                      Xerxes, Act 1: "Ombra mai fu"
3   4WnxBVhWBcM8z6GRscabSm                       Sialh Na Mban / Winnie's Air
4   0JwSo7FRo1bPRp2IP3kJgM                            Lehar Remix - Raag Desh
5   6anG7aR0p9enoBrwQybI4A                            Mèlange (Raag Bhimplas)
6   6ZilcYUIICcj8ihQ165irs                                           Powerman
7   0NoooClEgxqlqb4A5NBx9a                                Raga Aberi Track 02
8   4wcBlgP2YgOqoXzupVT8rb   Fantasy Pieces, Op. 12: Des Abends (The Evening)
9   5YsUCWWe7pdnJO43Yj7TFh                    Raga Hemant Gat In Drut Teental
10  67VWosPALzIS5jSoOrElFj                                             Mathey
11  3k0uRcdCErJnwup3sfjUX6      Deux arabesques (Two Arabesques), L 66: No. 1
12  7rZ5cv4pLF5X8EIe9D9jwS  Kinderszenen, Op. 15: Träumerei (Orchestra ver...
13  2eRxWvbXjEZYdLW5lErC3T  Typewriter, Tip, Tip, Tip from Merchant Ivory'...
14  5P0WyP7KPYTOZwnFPsqGjQ  Bonnie Banks Of Loch Lomond / Cam' Ye By Athol...
15  7tXLXpjVFwsNbmzgygvA8x  Concerto No. 1 in E-Flat Major for Piano and O...
16  2iDlp91egLSxpeuuYeLsqg          String Quintet in E Major, Op. 13: Minuet
17  2ExEChLCSorn4k3FUxs7td                     Where Do You Go To (My Lovely)
18  3sAqCSS70AIBmZitPV0iUX                       Varnam - Kedara Gowlai - Adi
19  01CWFykGVkAgqaq6mx4S9t      Two Elegiac Melodies, Op. 34: II. Last Spring
20  3q0wuJHbEtwRX5mVYXAQR5        Peer Gynt Suite No. 1, Op. 46: Morning Mood
21  16fPzsYDNAz5CphU8SNDM2                       Kanak Shaila - Punnagavarali
22  20YOTskpIGrtIUEcyt16xI                  Explorations In Vilambit Teentaal
23  7kWxCuG2ReGIMtjLe2CGav            Parvati Nayaka - Mohanam - Misra Tripta
24  14EYVKNIgesnKeaq92b8Zd                                        Quadrospect
25  1pVTwIeqARgzKllIyzqpk8  Concerto in E-Flat Major for Trumpet and Orche...
26  5xnlsjGhXul8b5dE7HnWg8                Valaci Vacci - Navaragamalika - Adi
27  7ecszxDX70AgUT6jNLTB7w                      Techno Wave (Raag Saraswathi)
28  5dJEx7XVIMB8H5yhrxzJkd                         Lahra & Jhala In Dipchandi
29  4fbSQbKoHNvIfqmwvb0xGJ                 Messiah, HWV 56: Hallelujah Chorus
30  4wBDr96bucxRIOzv1wPPd1  Three Fantasy Pieces for Cello and Piano, Op. ...
31  7ywbaVjKImedTvUguWQMih  Sonata No. 8 in C Minor for Piano, Op. 13, "Pa...
32  5yMnVl9geQTSzsIK2agXpl  Prelude No. 15 in D-Flat Major, Op. 28, "Raind...
33  5CnWRO3c4UWRwCkGxKO0Ly  Chorale for Organ, BWV 650: "Kommst du nun, Je...
34  1et1pAxMY8jFfblk9JIMvX  L'estro armonico, Op. 3 - Concerto No. 6 in A ...
35  17TYaKhyzETADo7SOoTTy5                     Ripples (Raag Amruthavarshini)
36  3Twm0yU9Hv64SVbCenU00c                                         Dadra Solo
37  3JQdFzEDxg8YUpxvGpnwxJ                                   Rupak Variations
38  2ZE2A47zah0C3Ee2uVKDbT                                 This Time Tomorrow
39  4bevNWkPkIvtQfZMJiysVP  Piano Sonata in A major, K. 331 - III. Finale ...
40  1GSKdr9KifUNGGLDfZ6hqh          Suite bergamasque, L.75: 3. Clair de lune
41  3L6bCxrwKe1qnsqFzssR4R                                          Strangers
42  1W6CAYXXYGYnq0xFV0CiZb  Concerto No. 2 in D Major for Cello and Orches...
43  2aiNUE9ia8e9WdUpVIcVYQ  Solomon, Act III: The Arrival of the Queen of ...
44  4h83R41t8oX0mctk2KLCBZ                           Mahanagar (The Big City)
45  1DWANIwKlOjnQ3Yn7gJmMe  Slåtter (Norwegian Peasant Dances), Op. 72: Br...
46  4yK7u3SViQC3Jhw4XUHkBC                     Amour (Raag Kalyana Vasantham)
47  79NwpxAm72bP8KKS8el0tD                                 Journey To Sixteen
48  28iwfJUUjKbiPEQ9eVx4XH                                Raga Aberi Track 04
49  2MRWXvnatHSK9S9JfXQQvp                 Jaane Kaise (feat. Shashwat Singh)